In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F


In [ ]:
model_name = "monologg/bert-base-cased-goemotions-original"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
emotion_labels = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval',
    'caring', 'confusion', 'curiosity', 'desire', 'disappointment',
    'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear',
    'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism',
    'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral'
]

In [ ]:
#
import pickle
import time
#
with open("pv_filtered.pkl", "rb") as f:
    pv_dict = pickle.load(f)
    texts = [pv_dict[key] for key in pv_dict.keys()]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
target_dict = pv_dict

result = {k: None for k in target_dict.keys()}
with torch.no_grad():
    for key in target_dict.keys():
        value = target_dict[key]
        inputs = tokenizer(value, return_tensors="pt", truncation=True, padding=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        try:
            outputs = model(**inputs)
            logits = outputs.logits
            probs = torch.sigmoid(logits)[0].cpu().numpy()
            result[key] = probs
        except Exception as e:
            print(e)

with open("pv_goemotions.pkl", "wb") as f:
    pickle.dump(result, f)